# ✅ Rolling Window MLP with Regularization & Early Stopping

In [ ]:

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


In [ ]:

# Load and preprocess
df = pd.read_csv(r"C:\Users\axxiv\Desktop\PI\mm-5G-prepared.csv")
df['Throughput_smoothed'] = df['Throughput'].rolling(window=5, min_periods=1).mean()
scaler = MinMaxScaler()
df['Throughput_smoothed_scaled'] = scaler.fit_transform(df[['Throughput_smoothed']])


In [ ]:

class RollingMLPDataset(Dataset):
    def __init__(self, series, input_len=48):
        self.series = series
        self.input_len = input_len

    def __len__(self):
        return len(self.series) - self.input_len

    def __getitem__(self, idx):
        x = self.series[idx:idx+self.input_len]
        y = self.series[idx+self.input_len]
        return torch.tensor(x, dtype=torch.float32), torch.tensor([y], dtype=torch.float32)

input_len = 48
series = df['Throughput_smoothed_scaled'].values
dataset = RollingMLPDataset(series, input_len)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=64)


In [ ]:

class SimpleMLP(nn.Module):
    def __init__(self, input_len, hidden_dim=64):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_len, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.4),  # Increased dropout
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.model(x)

model = SimpleMLP(input_len=input_len)


In [ ]:

# Loss, optimizer with weight decay, scheduler
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


In [ ]:

# Training with early stopping
train_losses, val_losses = [], []
best_val = float('inf')
patience = 5
counter = 0
best_weights = None

for epoch in range(50):
    model.train()
    batch_losses = []
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        batch_losses.append(loss.item())
    train_loss = np.mean(batch_losses)
    train_losses.append(train_loss)

    model.eval()
    with torch.no_grad():
        val_loss = np.mean([loss_fn(model(xb), yb).item() for xb, yb in val_dl])
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}: Train = {train_loss:.4f}, Val = {val_loss:.4f}")
    scheduler.step()

    if val_loss < best_val:
        best_val = val_loss
        best_weights = model.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

if best_weights:
    model.load_state_dict(best_weights)


In [ ]:

# Plot losses
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title("MLP with Regularization")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Predictions
model.eval()
xbatch, ytrue = next(iter(val_dl))
with torch.no_grad():
    ypred = model(xbatch).squeeze().numpy()

ytrue_inv = scaler.inverse_transform(ytrue.numpy().reshape(-1, 1)).flatten()
ypred_inv = scaler.inverse_transform(ypred.reshape(-1, 1)).flatten()

plt.figure(figsize=(12,5))
plt.plot(ytrue_inv, label='True Smoothed')
plt.plot(ypred_inv, label='Predicted Smoothed')
plt.title("Regularized MLP Prediction vs True")
plt.grid(True)
plt.legend()
plt.show()
